# Instagram Hashtag Poster

This notebook downloads all images from given hashtag and puts them into one large poster.

**Important info**: The Instagram API has a sanbox mode, which prevents any app created after Nov 2015 download images from all users prior to app approval. The workaround is to use an old Instagram app id - unfortunately after June 2016 this won't work too. See https://www.instagram.com/developer/sandbox/ for more info.

In [1]:
import math
import requests
from io import BytesIO
from IPython.display import Javascript
from PIL import Image

CLIENT_ID = '7b75033299b04b478a75b59044238b78'
REDIRECT_URI = 'http://localhost:8888/notebooks/notebooks/Instagram%20Hashtag%20Poster.ipynb'  # must be specified in ig client settings
IG_ENDPOINT = 'https://api.instagram.com/v1/'
IG_WIDTH = 320

THE_TAG = 'acupoftravel'

The two following cells handles OAuth2 authentication:

In [2]:
%%javascript
IPython.notebook.kernel.execute("NOTEBOOK_URL = '" + window.location.href + "'");

<IPython.core.display.Javascript object>

In [3]:
fragment = '#access_token='

if fragment in NOTEBOOK_URL:
    ACCESS_TOKEN = NOTEBOOK_URL[NOTEBOOK_URL.index(fragment)+len(fragment):]
    print('Got the token! Continue below.')
else:
    Javascript('location.href = "http://google.com"')
    print('Click the link and run the cells again.')
    print('https://api.instagram.com/oauth/authorize/?client_id={}&redirect_uri={}&response_type=token'.format(CLIENT_ID, requests.compat.quote_plus(REDIRECT_URI)))

Click the link and run the cells again.
https://api.instagram.com/oauth/authorize/?client_id=7b75033299b04b478a75b59044238b78&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Fnotebooks%2Fnotebooks%2FInstagram%2520Hashtag%2520Poster.ipynb&response_type=token


In [ ]:
# Init

r = requests.get(IG_ENDPOINT + 'tags/{}/'.format(THE_TAG), {'access_token': ACCESS_TOKEN})
n_images = r.json()['data']['media_count']
poster_width = math.floor(math.sqrt(n_images))
poster_img = Image.new('RGB', (IG_WIDTH*poster_width, IG_WIDTH*poster_width))

In [ ]:
# Download all images from the hashtag and place them on the poster. Beware, the poster may eat a lot of memory.

i = 0
r = requests.get(IG_ENDPOINT + 'tags/{}/media/recent'.format(THE_TAG), {'access_token': ACCESS_TOKEN})
while 'next_url' in r.json()['pagination']:
    for img_json in r.json()['data']:
        img_url = img_json['images']['low_resolution']['url']
        img = Image.open(BytesIO(requests.get(img_url).content))
        poster_img.paste(img, (IG_WIDTH*(i%poster_width), IG_WIDTH*(i//poster_width)))
        i += 1
        
        if i % poster_width == 0:
            print('Row {}/{}'.format(i//poster_width, poster_width))

    r = requests.get(r.json()['pagination']['next_url'])

In [ ]:
poster_img.save('ig_poster_{}.jpg'.format(THE_TAG), 'JPEG', quality=90, optimize=True, progressive=True)